#### Imports

In [1]:
import json
import nltk
from nltk.collocations import *
import datetime as dt
import locale
import spacy
from tqdm import tqdm
import pprint
import pandas as pd
from ast import literal_eval
import os
import gensim
import gensim.corpora as corpora
from pprint import pprint
from tqdm import tqdm 


In [2]:
!python -m spacy download de_core_news_lg

2023-05-03 14:42:21.432669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [3]:
#NOTE: default version doesntwork in colab
!pip install pyLDAvis==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97736 sha256=066686b6410d4e4f07a97cc7407c91db9bbaa45f65133b639647ac9c6d345bb8
  Stored in directory: /root/.cache/pip/wheels/d9/93/d6/16c95da19c32f037fd75135ea152d0df37254c25cd1a8b4b6c
Successfully built pyLDAvis


#### Read Data

In [4]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#change cwd
%cd drive/MyDrive/Work/Frontline/data
#%cd /content/drive/MyDrive/data/

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [6]:
# paths
FILTERED_PATH="filtered_4_26"

In [10]:
dfs = []

# loop through files 
for filename in tqdm(os.listdir(FILTERED_PATH)):
    # if csv file, load and add to dfs  
    if filename.endswith(".csv"):
        file_path = os.path.join(FILTERED_PATH, filename)
        df = pd.read_csv(file_path, index_col=0, converters={"text":literal_eval})
        dfs.append(df)

# combine files in df
df = pd.concat(dfs, ignore_index=True)

100%|██████████| 207/207 [00:15<00:00, 13.76it/s]


### Topic Modelling


#### Prepare Data


In [11]:
# custom module
import preprocessing

In [12]:
# Load model
spacy_mod = spacy.load("de_core_news_lg", disable=['ner', 'parser', 'tagger'])

In [13]:
# read custom stopwords

# open list of custom stopwords
custom_stop_words= open("custom_stopwords.txt").read().split()

# add custom stopwords to model
for word in custom_stop_words:
  spacy_mod.Defaults.stop_words.add(word)

In [16]:
smaller_df=df.iloc[:10000,:]

In [18]:
# convert corpus to language object
spacy_lang = []
for i, doc in tqdm(smaller_df.iterrows()): 
  spacy_lang.append(spacy_mod("".join(doc['text'])))

10000it [05:01, 33.16it/s]


In [19]:
# preprocess: remove stopwords
spacy_cleaned = []
for doc in tqdm(spacy_lang): 
    spacy_cleaned.append(preprocessing.preprocess(doc, remove_ent=True))

100%|██████████| 10000/10000 [00:05<00:00, 1974.04it/s]


#### Topic Analysis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import pickle  

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(spacy_cleaned)

# Create Corpus
texts = spacy_cleaned

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [21]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word, num_topics=num_topics)
# Print the key words for each topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"gewalt" + 0.008*"frauen" + 0.005*"kinder" + 0.004*"häuslicher" + '
  '0.004*"polizei" + 0.004*"menschen" + 0.004*"frau" + 0.004*"mann" + '
  '0.004*"opfer" + 0.003*"telefon"'),
 (1,
  '0.020*"telefon" + 0.014*"gewalt" + 0.012*"frauen" + 0.006*"häuslicher" + '
  '0.006*"bereitschaftsdienst" + 0.004*"beratung" + 0.004*"prozent" + '
  '0.004*"kinder" + 0.004*"polizei" + 0.004*"montag"'),
 (2,
  '0.030*"telefon" + 0.016*"bereitschaftsdienst" + 0.016*"gewalt" + '
  '0.012*"frauen" + 0.009*"häuslicher" + 0.006*"do" + 0.006*"montag" + '
  '0.006*"jugendtelefon" + 0.006*"beratung" + 0.005*"interventionsstelle"'),
 (3,
  '0.033*"telefon" + 0.017*"mi" + 0.016*"bereitschaftsdienst" + 0.015*"gewalt" '
  '+ 0.014*"do" + 0.008*"frauen" + 0.008*"häuslicher" + 0.006*"montag" + '
  '0.006*"jugendtelefon" + 0.006*"polizei"'),
 (4,
  '0.014*"telefon" + 0.010*"gewalt" + 0.009*"frauen" + 0.004*"frau" + '
  '0.004*"kinder" + 0.004*"menschen" + 0.004*"beratung" + 0.003*"donnerstag" + '
  '0.00

In [22]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.056123 -0.001430       1        1  35.570021
2      0.038649 -0.013374       2        1  20.861339
1     -0.004070 -0.006649       3        1  16.562720
4     -0.023657  0.008134       4        1  14.205388
3      0.045201  0.013319       5        1  12.800532, topic_info=                     Term          Freq         Total Category  logprob  \
152               telefon  19137.000000  19137.000000  Default  30.0000   
2396                   mi   3243.000000   3243.000000  Default  29.0000   
46    bereitschaftsdienst   8068.000000   8068.000000  Default  28.0000   
2394                   do   3470.000000   3470.000000  Default  27.0000   
2                  gewalt  13911.000000  13911.000000  Default  26.0000   
...                   ...           ...           ...      ...      ...   
3470                   fr    390.136114   1774.779881   Topic5  -5.9255   
54               dienstag    416.306336   2325.988424   Topic5  -5.8606   
112              mittwoch    427.852221   2503.435995   Topic5  -5.8333   
81                  hilfe    437.946557   2726.117181   Topic5  -5.8099   
435                kinder    484.407127   4771.975960   Topic5  -5.7091   

      loglift  
152   30.0000  
2396  29.0000  
46    28.0000  
2394  27.0000  
2     26.0000  
...       ...  
3470   0.5407  
54     0.3352  
112    0.2890  
81     0.2271  
435   -0.2319  

[443 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
3472       1  0.110217            abends
3472       2  0.283415            abends
3472       3  0.117215            abends
3472       4  0.069979            abends
3472       5  0.418125            abends
...      ...       ...               ...
27120      5  0.379757  ärzteallgemeiner
53384      3  0.492177     übereinstimmt
28437      1  0.904372         übergaben
8443       1  0.880175       überlebende
8443       5  0.088017       überlebende

[1101 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4])

END OF CODE